<a href="https://colab.research.google.com/github/ffmenezes/panorama_vacinacao_covid19/blob/main/analisa_dados_vacinacao_COVID19_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INTRO**

Notebook apresentando a forma de extração dos dados para construção do painel **[Vacina COVID-19: panorama atual](http://antiamador.org/bi/vacinaBR)**. Ao final, são feitas análises mais detalhadas, buscando dirimir algumas dúvidas acerca dos dados.

# **PRE**

In [ ]:
## importação das bibliotecas a serem utilizadas

In [ ]:
import os
import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import re
from unicodedata import normalize
import urllib
from bs4 import BeautifulSoup

# **PATH**

In [6]:
## configuro como diretório padrão uma pasta no meu Drive
os.chdir('/content/drive/Shareddrives/gdrive/datasets/minsaude/vacinacao_covid/')

# **FUNCTIONS**

In [8]:
## função para mostrar barra de progresso no download dos arquivos
class DownloadProgressBar(tqdm.tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

# **START**

## **Load Data**

### **Municípios**

In [ ]:
municipios = pd.merge(pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv', dtype={'codigo_ibge':'Int32','nome':'object','latitude':'float64','longitude':'float64','capital':'bool','codigo_uf':'Int8'}),
                      pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/estados.csv', header=0, names=['codigo_uf','sigla_uf','nome_uf','lat_uf','long_uf'], dtype={'codigo_uf':'Int8', 'sigla_uf':'object','nome_uf':'object','lat_uf':'float64','long_uf':'float64'})[['codigo_uf','sigla_uf','nome_uf']],
                      left_on='codigo_uf',right_on='codigo_uf')

In [ ]:
municipios.insert(1,'codigo_ibge_6', np.nan)
municipios['codigo_ibge_6'] = municipios.codigo_ibge.astype(str).str[:6].astype('int32')

In [ ]:
## agrega informação de estimativa populacional por município, para 2020, extraída do site do IBGE
municipios = municipios.merge(
    pd.read_csv('apoio/estim_pop_2020.csv', sep=';', skiprows=3, skipfooter=97, engine='python', usecols=['Cód.','2020']).rename(columns={'Cód.': 'codigo_ibge','2020': 'pop_estimada_2020'}),
    how='left', left_on = 'codigo_ibge', right_on='codigo_ibge')

In [ ]:
municipios.sample(5)

,codigo_ibge,codigo_ibge_6,nome,latitude,longitude,capital,codigo_uf,sigla_uf,nome_uf,pop_estimada_2020
2675,2612109,261210,Salgadinho,-7.92690,-35.6503,False,26,PE,Pernambuco,11068
5115,2703700,270370,Jaramataia,-9.66224,-37.0046,False,27,AL,Alagoas,5761
4184,3504800,350480,Bálsamo,-20.73480,-49.5865,False,35,SP,São Paulo,9139
4454,3528700,352870,Marabá Paulista,-22.10680,-51.9617,False,35,SP,São Paulo,5948
1479,2904308,290430,Brejões,-13.10390,-39.7988,False,29,BA,Bahia,14222


In [ ]:
# municipios.to_csv('processed/municipios.csv', index=False)

### **Registros de vacinação (OpenDataSUS)**

#### ***Download***

In [9]:
url_base = 'https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8'
b = BeautifulSoup(urllib.request.urlopen(url_base))

In [10]:
links = {link.text:link.get('href') for link in b.findAll('a', attrs={'href': re.compile(".csv$")})}
links

{'Dados AC': 'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-04-15/uf%3DAC/part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv',
 'Dados AL': 'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-04-15/uf%3DAL/part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv',
 'Dados AM': 'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-04-15/uf%3DAM/part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv',
 'Dados AP': 'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-04-15/uf%3DAP/part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv',
 'Dados BA': 'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-04-15/uf%3DBA/part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv',
 'Dados CE': 'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-04-15/uf%3DCE/part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv',
 'Dados Completos': 'https://s3-sa-east-1.amaz

In [11]:
## Para download do arquivo completo
for k,v in links.items():
  if 'Completos' in k:
    download_url(v,'{}.csv'.format(k.split(' ')[-1]))

part-00000-1c90f316-21cb-46b8-a2fb-57d03964b29d-c000.csv: 16.7GB [11:37, 24.0MB/s]                            


In [ ]:
## Para download dos arquivos por UF
for k,v in links.items():
  if 'Completos' not in k:
    download_url(v,'{}.csv'.format(k.split(' ')[-1]))

part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 57.0MB [00:05, 10.3MB/s]                            
part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 278MB [00:18, 15.0MB/s]                           
part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 291MB [00:27, 10.5MB/s]                           
part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 52.3MB [00:05, 10.0MB/s]                            
part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 1.43GB [01:14, 19.1MB/s]                            
part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 654MB [00:38, 17.0MB/s]                           
part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 189MB [00:13, 13.9MB/s]                           
part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 365MB [00:21, 16.7MB/s]                           
part-00000-5476c5fc-7a69-407c-abce-9eebb0134b29.c000.csv: 485MB [00:27, 17.9MB/s]                           
part-00000-54

**Há diferença entre os arquivos 'completo' e por UF?**

In [16]:
soma_linhas = {}

for file in tqdm.tqdm_notebook(os.listdir()):
  if '.csv' in file:
    with open(file) as f:
        soma_linhas[file.split('.')[0]] = sum(1 for line in f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
print(sum([v-1 for k,v in soma_linhas.items() if k != 'Completos']),'linhas de registros nos arquivos por UF')
print(soma_linhas['Completos']-1,'linhas de registros no arquivo completo.')

31189012 linhas de registros nos arquivos por UF
31189012 linhas de registros no arquivo completo.


**Ótimo, como não há divergência quantitativa entre os arquivos, a decisão de qual utilizar será puramente pela perfomance. Vamos fazer agregações por arquivos de UFs então.**

#### ***Generate AGGR data***

In [ ]:
## filtra apenas os arquivos "UF.csv" da pasta
lista_arquivos = [file for file in os.listdir() if len(file.split('.')[0]) == 2]

In [ ]:
## iteração para ler cada arquivo, selecionando apenas algumas colunas e fazendo tratamentos básicos
for file in tqdm.tqdm_notebook(lista_arquivos):

  df_vacinas = pd.read_csv(file, sep=';', encoding='utf8',
                          usecols=[
                                    'document_id', 'paciente_id', 
                                    # 'paciente_idade', 
                                    # 'paciente_dataNascimento', 'paciente_enumSexoBiologico', 
                                    #'paciente_racaCor_codigo', 
                                    # 'paciente_racaCor_valor', 
                                    'paciente_endereco_coIbgeMunicipio', 
                                    # 'paciente_endereco_coPais', 'paciente_endereco_nmMunicipio', 'paciente_endereco_nmPais', 'paciente_endereco_uf', 'paciente_endereco_cep', 'paciente_nacionalidade_enumNacionalidade', 
                                    # 'estabelecimento_valor', 
                                    # 'estabelecimento_razaoSocial', 'estalecimento_noFantasia', 
                                    'estabelecimento_municipio_codigo', 
                                    # 'estabelecimento_municipio_nome', 'estabelecimento_uf', 
                                    # 'vacina_grupoAtendimento_codigo', 
                                    'vacina_grupoAtendimento_nome',
                                    # 'vacina_categoria_codigo', 
                                    'vacina_categoria_nome',
                                    # 'vacina_lote', 'vacina_fabricante_nome', 'vacina_fabricante_referencia', 
                                    'vacina_dataAplicacao', 'vacina_descricao_dose',
                                    # 'vacina_codigo', 
                                    'vacina_nome', 
                                    # 'sistema_origem', 
                                    'data_importacao_rnds'
                                    ],
                          parse_dates=['vacina_dataAplicacao', 'data_importacao_rnds'],
                          dtype= {'paciente_endereco_coIbgeMunicipio': 'Int32', 'vacina_categoria_codigo':'Int8', 'vacina_grupoAtendimento_codigo':'Int16', 'estabelecimento_municipio_codigo': 'Int32'},
                          )
  
  ## marcador da UF na primeira coluna
  df_vacinas.insert(0, 'UF', file.split('.')[0])
  
  ## elimina linhas duplicadas
  df_vacinas.drop_duplicates(subset='document_id', keep='last', inplace=True)
  
  ## normaliza e limpa o campo de descrição da dose, eliminando espaços excedentes
  df_vacinas.vacina_descricao_dose = df_vacinas.vacina_descricao_dose.apply(lambda x: normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII').strip())
  
  ## cria um contador de cada 1ª dose aplicada
  df_vacinas['contador_1a_dose'] = np.where(df_vacinas.vacina_descricao_dose == '1a Dose', 1, 0)
  
  ## cria um contador de cada 2ª dose (ou dose única) aplicada
  df_vacinas['contador_2a_dose'] = np.where((df_vacinas.vacina_descricao_dose == '2a Dose') | (df_vacinas.vacina_descricao_dose == 'Unica'), 1, 0)

  """
  Cria um "contador de dose aplicada" para cada linha. 
  ATENÇÃO! A soma dos contadores anteriores não fecharão com este, pois há descrições inseridas que fogem do padrão, como por exemplo: "Dose". 
  A priori, não se tem como identificar se é a única, a primeira, a segunda, a terceira... pois não segue um padrão. Análise deve ser feita com cada arquivo e cada paciente.
  Para o painel de BI, apenas os contadores iniciais são utilizados, descartando os casos não registrados corretamente, que são uma pequeníssima parcela.
  """
  df_vacinas.insert(0, 'contador', 1)

  ## formatação da data de aplicação, eliminando hora
  df_vacinas.vacina_dataAplicacao = df_vacinas.vacina_dataAplicacao.dt.date
  df_vacinas.vacina_dataAplicacao = pd.to_datetime(df_vacinas.vacina_dataAplicacao)

  ## faz correção manual quando data aplicação é maior que data registro
  df_vacinas['vacina_dataAplicacao'] = np.where(df_vacinas['vacina_dataAplicacao'] > df_vacinas['data_importacao_rnds'], df_vacinas['data_importacao_rnds'], df_vacinas['vacina_dataAplicacao'])
  df_vacinas.vacina_dataAplicacao = df_vacinas.vacina_dataAplicacao.dt.date
  df_vacinas.vacina_dataAplicacao = pd.to_datetime(df_vacinas.vacina_dataAplicacao)

  ## quando município de endereço do paciente não é preenchido, ou é valor não identificado na base oficial, considera o município do estabelecimento como o de residência
  df_vacinas.paciente_endereco_coIbgeMunicipio = df_vacinas.paciente_endereco_coIbgeMunicipio.map(municipios.set_index('codigo_ibge_6').codigo_ibge.to_dict())
  df_vacinas.estabelecimento_municipio_codigo = df_vacinas.estabelecimento_municipio_codigo.map(municipios.set_index('codigo_ibge_6').codigo_ibge.to_dict())
  df_vacinas['paciente_endereco_coIbgeMunicipio'] = df_vacinas['paciente_endereco_coIbgeMunicipio'].fillna(df_vacinas['estabelecimento_municipio_codigo'])
  df_vacinas.paciente_endereco_coIbgeMunicipio = df_vacinas.paciente_endereco_coIbgeMunicipio.astype('Int32')

  ## faz agregação para contar número de doses aplicadas por dia, com base em determinadas características, e exporta para CSV da UF
  pd.concat([df_vacinas.groupby(['paciente_endereco_coIbgeMunicipio', 'vacina_dataAplicacao', 'vacina_categoria_nome', 'vacina_grupoAtendimento_nome', 'vacina_nome']).contador.sum(),
            df_vacinas.groupby(['paciente_endereco_coIbgeMunicipio', 'vacina_dataAplicacao', 'vacina_categoria_nome', 'vacina_grupoAtendimento_nome', 'vacina_nome']).contador_1a_dose.sum(),
            df_vacinas.groupby(['paciente_endereco_coIbgeMunicipio', 'vacina_dataAplicacao', 'vacina_categoria_nome', 'vacina_grupoAtendimento_nome', 'vacina_nome']).contador_2a_dose.sum()],
            axis=1).to_csv('processed/aggr_doses_{}.csv'.format(file.split('.')[0]), header=False)

  ## marca qual a data/hora mais recente de importação no RNDS de cada UF
  df_vacinas.groupby('UF').data_importacao_rnds.max().reset_index().to_csv('processed/max_date.csv', mode='a', header=False, index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


# **ANALYSIS**
Acima, foi o procedimento básico para gerar os dados agregados vistos no [painel](http://antiamador.org/bi/vacinaBR).<br>

Abaixo, vou pegar um (ou alguns) arquivos como parâmetro e fazer uma análise mais detalhada, **por meio de questionamentos**, para apresentar alguns pontos incongruentes dos dados, ou pelo menos, intrigantes. Também, detalhar a motivação dos tratamentos realizados na agregação. <br>

Aproveito para reforçar o link da fonte das informações, inclusive com a metodologia para consumo via API, e o dicionário de dados: https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao

## **LOAD**

In [18]:
file = 'RS.csv'

In [19]:
df_vacinas = pd.read_csv(file, sep=';', encoding='utf8',
                          usecols=[
                                    'document_id', 
                                    'paciente_id', 'paciente_idade', 'paciente_dataNascimento', 'paciente_enumSexoBiologico', 'paciente_racaCor_codigo', 'paciente_racaCor_valor', 'paciente_endereco_coIbgeMunicipio', 'paciente_endereco_coPais', 'paciente_endereco_nmMunicipio', 'paciente_endereco_nmPais', 'paciente_endereco_uf', 'paciente_endereco_cep', 'paciente_nacionalidade_enumNacionalidade', 
                                    'estabelecimento_valor', 'estabelecimento_razaoSocial', 'estalecimento_noFantasia', 'estabelecimento_municipio_codigo', 'estabelecimento_municipio_nome', 'estabelecimento_uf', 
                                    'vacina_grupoAtendimento_codigo', 'vacina_grupoAtendimento_nome', 'vacina_categoria_codigo', 'vacina_categoria_nome', 
                                    'vacina_lote', 'vacina_fabricante_nome', 'vacina_fabricante_referencia', 
                                    'vacina_dataAplicacao', 'vacina_descricao_dose',
                                    'vacina_codigo', 'vacina_nome', 
                                    'sistema_origem', 'data_importacao_rnds'
                                    ],
                          parse_dates=['vacina_dataAplicacao', 'data_importacao_rnds', 'paciente_dataNascimento'],
                          dtype= {'paciente_endereco_coIbgeMunicipio': 'Int32', 
                                  'vacina_categoria_codigo':'Int8', 
                                  'vacina_grupoAtendimento_codigo':'Int16', 
                                  'estabelecimento_municipio_codigo': 'Int32'},
                          )

In [23]:
df_vacinas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329087 entries, 0 to 2329086
Data columns (total 33 columns):
 #   Column                                    Dtype              
---  ------                                    -----              
 0   document_id                               object             
 1   paciente_id                               object             
 2   paciente_idade                            int64              
 3   paciente_dataNascimento                   datetime64[ns]     
 4   paciente_enumSexoBiologico                object             
 5   paciente_racaCor_codigo                   int64              
 6   paciente_racaCor_valor                    object             
 7   paciente_endereco_coIbgeMunicipio         Int32              
 8   paciente_endereco_coPais                  float64            
 9   paciente_endereco_nmMunicipio             object             
 10  paciente_endereco_nmPais                  object             
 11  paciente_en

In [24]:
df_vacinas.sample(5)

,document_id,paciente_id,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_coIbgeMunicipio,paciente_endereco_coPais,paciente_endereco_nmMunicipio,paciente_endereco_nmPais,paciente_endereco_uf,paciente_endereco_cep,paciente_nacionalidade_enumNacionalidade,estabelecimento_valor,estabelecimento_razaoSocial,estalecimento_noFantasia,estabelecimento_municipio_codigo,estabelecimento_municipio_nome,estabelecimento_uf,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds
136417,18880fec-b6f3-4cef-852d-f3ea025fa248-i0b0,1c45003daee9007f9d82d324f3f7799d74254d167103af...,58,1962-11-22,F,2,PRETA,431225,10.0,MINAS DO LEAO,BRASIL,RS,96755.0,B,2226693,PREFEITURA MUNICIPAL DE MINAS DO LEAO,CENTRO MUNICIPAL DE SAUDE MINAS DO LEAO,431225,MINAS DO LEAO,RS,905,Cuidador de Idosos,9,Trabalhadores de Saúde,200279,FUNDACAO BUTANTAN,Organization/61189445000156,2021-02-09 03:00:00+00:00,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-02-13 01:19:33
969601,c345270e-7857-4c5e-ab33-70b3a44744b6-i0b0,f252e888adf3e1426d51335b1b00fa0f308f9b9510121b...,64,1956-06-13,F,1,BRANCA,431490,10.0,PORTO ALEGRE,BRASIL,RS,91430.0,B,7250479,PREFEITURA MUNICIPAL DE PORTO ALEGRE,GERENCIA DISTRITAL CENTRO,431490,PORTO ALEGRE,RS,905,Cuidador de Idosos,9,Trabalhadores de Saúde,4120Z005,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-02-25 03:00:00+00:00,1ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-03-16 13:22:54
1662095,f65a3912-880c-4c4e-ae9b-feba2e18b4e2-i0b0,dd150afa4f7f275bc932e299d75b4829d950fc968c58d9...,73,1947-09-26,M,4,AMARELA,432215,10.0,TUNAS,BRASIL,RS,99330.0,B,2245604,PREFEITURA MUNICIPAL DE TUNAS,CENTRO DE SAUDE DE TUNAS TUNAS,432215,TUNAS,RS,203,Pessoas de 70 a 74 anos,2,Faixa Etária,210075,FUNDACAO BUTANTAN,Organization/61189445000156,2021-03-26 03:00:00+00:00,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-03-28 20:50:59
273561,d53e3e91-fbba-42aa-9c6a-e0d9a2c550ee-i0b0,0669f98657314182f265677a6a82324d8acc66b279b4f6...,65,1955-05-18,M,1,BRANCA,430820,10.0,FLORES DA CUNHA,BRASIL,RS,95270.0,B,2240289,PREFEITURA MUNICIPAL DE FLORES DA CUNHA,CENTRO DE SAUDE MUNICIPAL IRMA BENEDITA ZORZI,430820,FLORES DA CUNHA,RS,202,Pessoas de 65 a 69 anos,2,Faixa Etária,210134,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-03 03:00:00+00:00,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-05 20:05:05
2067964,ca06d144-9113-4f4a-8750-4a3836684926-i0b0,5117eba4f5cd322a81f37b1b58cc233c226dbf10f5f808...,61,1960-03-04,M,1,BRANCA,431440,10.0,PELOTAS,BRASIL,RS,96080.0,B,433489,MUNICIPIO DE PELOTAS,CENTRAL REDE DE FRIO MUNICIPL NUCLEO MUNCIPAL,431440,PELOTAS,RS,201,Pessoas de 60 a 64 anos,2,Faixa Etária,213VCD006ZVA,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-04-11 03:00:00+00:00,1ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-04-13 13:12:46


## **QUESTION**

### Há dados duplicados?
Sim. Ao que tudo indica, o 'Identificador do Documento' (`document_id`) é a chave única da base. Deveria então, representar uma vacinação exclusivamente. Em alguns arquivos de determinados Estados, não é o que acontece. Podemos ver inclusive que todas informações são idênticas.

In [25]:
df_vacinas.loc[df_vacinas.duplicated(subset='document_id',keep=False)].sort_values(by='document_id')[:10]

,document_id,paciente_id,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_coIbgeMunicipio,paciente_endereco_coPais,paciente_endereco_nmMunicipio,paciente_endereco_nmPais,paciente_endereco_uf,paciente_endereco_cep,paciente_nacionalidade_enumNacionalidade,estabelecimento_valor,estabelecimento_razaoSocial,estalecimento_noFantasia,estabelecimento_municipio_codigo,estabelecimento_municipio_nome,estabelecimento_uf,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds
1689853,00022e15-e1d5-4366-a9bf-fe5c483866a9-i0b0,10d322daa792f30230e2f4c15d9222bff146fbad1ce605...,76,1945-01-13,F,1,BRANCA,431320,10.0,NOVA PETROPOLIS,BRASIL,RS,95150.0,B,2238357,PREFEITURA MUNICIPAL DE NOVA PETROPOLIS,UNIDADE BASICA DE SAUDE CENTRO,431320,NOVA PETROPOLIS,RS,204,Pessoas de 75 a 79 anos,2,Faixa Etária,210051,SERUM INSTITUTE OF INDIA LTD,NaN,2021-03-12 00:00:00+00:00,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-03-22 00:39:30
1277332,00022e15-e1d5-4366-a9bf-fe5c483866a9-i0b0,10d322daa792f30230e2f4c15d9222bff146fbad1ce605...,76,1945-01-13,F,1,BRANCA,431320,10.0,NOVA PETROPOLIS,BRASIL,RS,95150.0,B,2238357,PREFEITURA MUNICIPAL DE NOVA PETROPOLIS,UNIDADE BASICA DE SAUDE CENTRO,431320,NOVA PETROPOLIS,RS,204,Pessoas de 75 a 79 anos,2,Faixa Etária,210051,SERUM INSTITUTE OF INDIA LTD,NaN,2021-03-12 00:00:00+00:00,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-03-22 00:39:30
395627,000e4e5e-03ba-4dbd-b1f4-c6985d3d2b99-i0b0,1b8789acd40fbf5c0789eef14ae728f38f8f683da344ab...,68,1952-07-30,M,1,BRANCA,430544,10.0,CHUVISCA,BRASIL,RS,96193.0,B,2224844,PREFEITURA MUNICIPAL DE CHUVISCA,POSTO DE SAUDE,430544,CHUVISCA,RS,202,Pessoas de 65 a 69 anos,2,Faixa Etária,210086,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-07 03:00:00+00:00,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-07 17:05:46
1493443,000e4e5e-03ba-4dbd-b1f4-c6985d3d2b99-i0b0,1b8789acd40fbf5c0789eef14ae728f38f8f683da344ab...,68,1952-07-30,M,1,BRANCA,430544,10.0,CHUVISCA,BRASIL,RS,96193.0,B,2224844,PREFEITURA MUNICIPAL DE CHUVISCA,POSTO DE SAUDE,430544,CHUVISCA,RS,202,Pessoas de 65 a 69 anos,2,Faixa Etária,210086,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-07 03:00:00+00:00,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-07 17:05:46
1352960,000fd77c-e7e1-4752-92b6-b505702c1b3d-i0b0,a80450816a9cbf69459088ed0bd646bf1cdce76fb4480a...,58,1962-03-29,F,5,INDIGENA,430885,10.0,GENTIL,BRASIL,RS,99160.0,B,6177743,MINISTERIO DA SAUDE,POLO BASE BARRA DO RIBEIRO,430190,BARRA DO RIBEIRO,RS,701,Povos indígenas em terras indígenas,7,Povos Indígenas,202010020,FUNDACAO BUTANTAN,Organization/61189445000156,2021-03-22 03:00:00+00:00,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-03-23 19:06:49
1627228,000fd77c-e7e1-4752-92b6-b505702c1b3d-i0b0,a80450816a9cbf69459088ed0bd646bf1cdce76fb4480a...,58,1962-03-29,F,5,INDIGENA,430885,10.0,GENTIL,BRASIL,RS,99160.0,B,6177743,MINISTERIO DA SAUDE,POLO BASE BARRA DO RIBEIRO,430190,BARRA DO RIBEIRO,RS,701,Povos indígenas em terras indígenas,7,Povos Indígenas,202010020,FUNDACAO BUTANTAN,Organization/61189445000156,2021-03-22 03:00:00+00:00,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-03-23 19:06:49
1112779,0010ec62-bdb5-4ee2-80a5-855f7362732e-i0b0,d47dbdf8d96a024b71ec550f9028eee59ce045b741e7f0...,63,1958-02-23,M,1,BRANCA,431500,10.0,PORTO LUCENA,BRASIL,RS,98980.0,B,2250268,PREFEITURA MUNICIPAL DE PORTO LUCENA,POSTO DE SAUDE PSF 1,431500,PORTO LUCENA,RS,201,Pessoas de 60 a 64 anos,2,Faixa Etária,210132,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-06 03:00:00+00:00,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-07 19:06:45
2

In [26]:
df_vacinas.shape

(2329087, 33)

In [27]:
df_vacinas.drop_duplicates(subset='document_id', keep='last', inplace=True)

In [28]:
df_vacinas.shape

(2318382, 33)

### Quem foi o/a paciente #0?

### Quem foi o paciente mais velho?

### E o mais novo?

### Quais os tipo de doses aplicadas?

### Algum paciente tomou mais de duas doses?

### Quantos estão respeitando o tempo para tomar a 2ª dose?

### Alguém tomou dois (ou mais) tipos de vacina?